In [12]:
# Main directory of the project
cd /Users/julia/bacterialGrowth_thesis/

# Create the directory in which all the intermediate files will be placed
mkdir -p IntermediateFiles/

# Unzip the given data
unzip -oq DataParsed.zip
rm -r __MACOSX

In [13]:
ls -lt

total 1280
drwxr-xr-x@ 112 julia  staff    3584  9 mar 16:10 DataParsed
drwxr-xr-x    7 julia  staff     224  9 mar 16:07 Notebooks
-rw-r--r--    1 julia  staff     832  8 mar 19:36 commands.md
drwxr-xr-x   12 julia  staff     384  8 mar 19:35 Python_scripts
drwxr-xr-x    7 julia  staff     224  8 mar 19:35 DB_scripts
drwxr-xr-x    8 julia  staff     256  8 mar 19:33 00_old
drwxr-xr-x   11 julia  staff     352  8 mar 19:32 IntermediateFiles
drwxr-xr-x   34 julia  staff    1088  7 mar 16:56 Data
-rwxr-xr-x    1 julia  staff      19 16 nov 20:48 README.md
-rwxr-xr-x@   1 julia  staff  645442 20 oct 09:36 DataParsed.zip


# 1) Get files names

* This BASH script parses the DataParsed directory and saves the names of the .txt files into `lab_files_names.txt`.

In [15]:
ls -lt  DataParsed/ | grep -i .txt | awk '{print $9}' > IntermediateFiles/lab_files_names.txt

## 1.a) Modifications in files names

For the moment I will just leave the lab_files_names_modified.txt as I used it in the following scripts. But I will not modify anything, as I had incorrectly assumptions when I modified them.

# 2) Get experiment names

* Extract the different experiment names fro the given directory. Several files belong to the same experiment, but we have one file per replicate.

This BASH script gets the modified names and then, by removing the last 6 characters, which are 4 for the extension and 2 for the number of replicates of one experiment, creates a file with the unique names of the experiments. These will be saved in `lab_experiment_names.txt`.
This will be useful when we try to classify the experiments depending on the metadata contained in the names the lab gave to each file. 
Also, in this classification, we will make directories with the names of the experiments that will be gotten from here.


In [26]:
cd /Users/julia/bacterialGrowth_thesis/IntermediateFiles

In [28]:
awk '{print substr($1,1,length($1)-6)}' lab_files_names_modified.txt | sort | uniq > lab_experiment_names.txt

In [29]:
wc -l lab_experiment_names.txt

      32 lab_experiment_names.txt


# 3) Get files headers

* Get the headers of the files. These fields correspond to the time, the pH measurement, bacterial abundances and metabolites

This BASH script goes through the `lab_files_names.txt` to get the true files names and look inside each of them. Each replicate has different (more or less) headers, so this script gets all of them (`lab_files_headers.txt`) and keeps the unique ones. The unique ones are contained in `lab_headers.txt`.


In [31]:
cd /Users/julia/bacterialGrowth_thesis/
rm IntermediateFiles/lab_files_headers.txt

while read -r file;
    do
        head -n 1 DataParsed/$file >> IntermediateFiles/lab_files_headers.txt
done < IntermediateFiles/lab_files_names.txt

In [33]:
cat lab_files_headers.txt | tr " " "\n" | sort | uniq > lab_headers.txt

# 4) Obtain experiment information

* Create an information file containing the name of each file, the experiment it belongs to and the number of replicate.
* Include more fields later!!!

Files must be name in the following way: `[experimentName]_[numberReplicate].txt` where `[experimentName]` is the name that will be placed in the database and `[numberReplicate]` the number of technical replicate for one concrete experiment.

For the moment, no information will be taken from the experiment name given in the lab in order to populate the DB.

In [36]:
rm experiments_info.txt

echo 'file_name,experiment_name,replicate_number' | column -t -s "," > experiments_info.txt

while read -r file; do

    file_name=$file
    experiment_name=${file_name:0:${#file_name}-6}
    replicate_number=${file_name:${#file_name}-5:1}
    echo $file_name $experiment_name $replicate_number >> experiments_info.txt
    
done < lab_files_names.txt

more experiments_info.txt | wc -l

     108


In [38]:
tail -n +2 experiments_info.txt > experiments_info_mod.txt

# 5) Create directories

This BASH script creates all the directories in which the replicates’ files are going to be placed. It takes the `experiments_info_mod.txt` file and gets the columns with the experiment name and the replicate number. It first creates a directory in the following way:

`/Users/julia/bacterialGrowth_thesis/Data/$experiment/$replicate`


In [75]:
cd /Users/julia/bacterialGrowth_thesis/
mkdir -p Data/
while read -r line; do

    experiment=$(echo $line | awk '{print $2}')
    replicate=$(echo $line | awk '{print $3}')
    
    file=$(echo $line | awk '{print $1}')
    
    cd /Users/julia/bacterialGrowth_thesis/Data
    mkdir -p $experiment
    mkdir -p $experiment/$replicate

    path_origin=/Users/julia/bacterialGrowth_thesis/DataParsed/"$file"
    path_destination=/Users/julia/bacterialGrowth_thesis/Data/$experiment/$replicate/
    
    cp $path_origin $path_destination

done < IntermediateFiles/experiments_info_mod.txt